In [43]:
import torch
import os
import json
import requests
import numpy as np

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import AutoProcessor, Pix2StructForConditionalGeneration

In [46]:
IMAGE_PATH = "data/images/"
QA_PATH = "data/qa/"

In [150]:
class RealCQA(Dataset):
    def __init__(self, img_list) -> None:
        super().__init__()
        self.img_list = img_list
        
    def __len__(self):
        return len(self.img_list)
    

    def __getitem__(self, idx):
        sample_id = self.img_list[idx][:-3]

        # Get image with following name
        image = Image.open(IMAGE_PATH + sample_id + 'jpg')
        with open(QA_PATH + sample_id + 'json', encoding='utf8') as f:
            qa = json.load(f)
        
        # Since every image has a plethora of questions, select one from them randomly
        rnd_sample = np.random.randint(len(qa))

        q, a = qa[rnd_sample]['question'], qa[rnd_sample]['answer']

        if isinstance(a, list):
            print(a)
            print(qa[rnd_sample]['answer_type'])
            a = ', '.join([el for el in a])
        
        return image, q, a, sample_id

In [151]:
imgs_list = os.listdir(IMAGE_PATH)

In [152]:
ds = RealCQA(imgs_list)

In [153]:
dataloader = DataLoader(ds, batch_size=32, shuffle=True)

In [154]:
next(iter(dataloader))

['Deer protection', 'Rodents protection', 'Deer reduction', 'Fencing', 'Rodents vaccination', 'Acaricides', 'Landscaping']
Unranked List


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>

In [94]:
model = Pix2StructForConditionalGeneration.from_pretrained("google/matcha-chartqa").to(0)
processor = AutoProcessor.from_pretrained("google/matcha-chartqa")

c:\Users\foret\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [95]:
inputs = processor(images=sample[0], text=sample[1], return_tensors="pt").to(0)
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

Real-Time Normal


In [157]:
inputs.keys()

dict_keys(['flattened_patches', 'attention_mask'])